In [1]:

import os, sys, math, time
import numpy as np
import numpy.linalg as la
import plotly.graph_objects as go
import plotly.express as ex
from plotly.subplots import make_subplots
import pandas as pd

import json as js
import _pickle as pickle
import bz2
import ray

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from collections import OrderedDict

sys.path.append("../")
import func


## Analyse features

In [5]:
data_path = "../../data/"
# load data
data = func.load(data_path+"LOCO_R2-default-locomotion.pbz2")
data_2 = func.load(data_path+"LOCO_R2-default-locomotion-small.pbz2")
data_3 = func.load(data_path+"LOCO_R2-default-locomotion-large.pbz2")

In [12]:
d1 = pickle.loads(data[0])
d2 = pickle.loads(data_2[0])
print("Target positions: ")
for pos in d1["targetPos"]:   print("1: ", pos)
for pos in d2["targetPos"]:   print("2: ", pos)
print("-"*70)

print("Target rotations: ")
for rot in d1["targetRot"]:   print("1: ", rot)
for rot in d2["targetRot"]:   print("1: ", rot)
print("-"*70)

Target positions: 
1:  [-0.08207791 -0.95545423 -0.09593999]
1:  [ 0.08207785 -0.7766549  -0.05018108]
2:  [-0.04103896 -0.47772712 -0.05297   ]
2:  [ 0.04103893 -0.12012845 -0.00721061]
----------------------------------------------------------------------
Target rotations: 
1:  [0. 0. 0.]
1:  [0. 0. 0.]
1:  [0. 0. 0.]
1:  [0. 0. 0.]
----------------------------------------------------------------------


In [19]:
pos1 = []
for f in d1["frames"]:
    p = [jo["pos"] for jo in f]
    pos1.append(p)

pos2 = []
for f in d2["frames"]:
    p = [jo["pos"] for jo in f]
    pos2.append(p)


posJ = []
for i in range(len(pos1[0])):
    p = [f[i] for f in pos1]
    posJ.append(p)

posJ2 = []
for i in range(len(pos2[0])):
    p = [f[i] for f in pos2]
    posJ2.append(p)


In [32]:
# plot position for R2 default
fig = make_subplots(rows=3, cols=1, subplot_titles=["X", "Y", "Z"])
x = np.arange(len(pos1))
i = 0
for j in posJ:
    y = [t[0] for t in j]
    y1 = [t[1] for t in j]
    y2 = [t[2] for t in j]
    fig.add_trace(go.Scatter(x=x, y=y, name="J{}".format(i), mode="lines"), row=1, col=1)
    fig.add_trace(go.Scatter(x=x, y=y1, name="J{}".format(i), mode="lines"), row=2, col=1)
    fig.add_trace(go.Scatter(x=x, y=y2, name="J{}".format(i), mode="lines"), row=3, col=1)
    i+=1

fig.update_layout(title="R2 DEFAULT")
fig.show()

In [33]:
# plot position for R2 small
fig = make_subplots(rows=3, cols=1, subplot_titles=["X", "Y", "Z"])
x = np.arange(len(pos2))
i = 0
for j in posJ2:
    y = [t[0] for t in j]
    y1 = [t[1] for t in j]
    y2 = [t[2] for t in j]
    fig.add_trace(go.Scatter(x=x, y=y, name="J{}".format(i), mode="lines"), row=1, col=1)
    fig.add_trace(go.Scatter(x=x, y=y1, name="J{}".format(i), mode="lines"), row=2, col=1)
    fig.add_trace(go.Scatter(x=x, y=y2, name="J{}".format(i), mode="lines"), row=3, col=1)
    i+=1
fig.update_layout(title="R2 SMALL")
fig.show()

## Simple MLP Autoencoder
$
f(x,\theta) = dec(enc(x,\theta_1), \theta_2) = x,   \quad \theta = (\theta_1, \theta_2)
$

$
enc(x, \theta_1) = z, \quad   z \in Z \quad \text{ = latent space}
$

$
dec(z, \theta_2) = x, \quad   x \in X \quad \text{ = input space}
$

This model uses simple Multi-layered perceptron (MLP) for both encoder and decoder.

$
enc = dec = mlp(X, \theta), \quad \theta = W, b

$
mlp(X, W) = f(f(X \cdot w_1 + b_1) \cdot w_2 + b_2) \cdot w_3 + b_3
$

In [3]:
class MLP(nn.Module):
    def __init__(self, dimensions:list, act_fn:str, keep_prob:float=.2, batch_size:int=1):
        super(MLP, self).__init__()
        self.dimensions = dimensions          #   [(in, h1), (h1, h2), ..., (hn, out)]
        self.act= act_fn                     #   func
        self.keep_prob = keep_prob          #   %
        self.batch_size = batch_size        #   int

        self.model = []

        assert(len(dimensions) >= 2)
        assert(batch_size > 0)
        assert(act_fn == "elu" or act_fn == "relu")
        assert(keep_prob < 1)
        for e in dimensions: assert(type(e) == int)

        self.build()
        self.model.apply(self.init_params)


    def build(self):
        layers = []
        for i, size in enumerate(zip(self.dimensions[0:], self.dimensions[1:])):
            layers.append(("fc"+str(i), nn.Linear(size[0], size[1])))
            if i < len(self.dimensions)-2:
                layers.append(("act"+str(i), self.activation(self.act)))
                layers.append(("drop"+str(i+1), nn.Dropout(self.keep_prob)))

        self.model = nn.Sequential(OrderedDict(layers))


    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.model(x)

    @staticmethod
    def activation(fn_name):
        if fn_name == "elu":
            return nn.ELU()
        elif fn_name == "relu":
            return nn.ReLU()
        else:
            return nn.ReLU()

    @staticmethod
    def init_params(m):
        if type(m) == nn.Linear:
            nn.init.xavier_normal_(m.weight)
            m.bias.data.fill_(.01)





In [4]:
class MLP_AE(nn.Module):
    def __init__(self, encoder:nn.Module, decoder:nn.Module):
        super(MLP_AE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder


    def forward(self, x):
        return self.decoder(self.encoder(x))

In [11]:
# Prepare train data
all_data = []
data = [data, data_2, data_3]

for da in data:
    for d in da:
        d = pickle.loads(d)
        features = []
        for f in d["frames"]:
            p = np.concatenate(
                [
                    np.concatenate([jo["pos"] for jo in f]),
                    np.concatenate([jo["rotMat"].ravel() for jo in f]),
                    np.concatenate([jo["velocity"] for jo in f]),
                 ])
            features.append(p)
        all_data.append(np.vstack(features))

# input_data = np.array([np.vstack([p for p in j]) for d in all_data for j in d])
input_data = np.asarray(all_data)
input_data.reshape((-1, input_data.shape[-1]))
print(input_data.shape)

(36, 120, 315)


In [16]:
data_ratio = (.7, .15, .15) # training, validation, testing
SEED = 2021
batch_size = 10

x_tensor = torch.from_numpy(input_data).float()
y_tensor = torch.from_numpy(input_data).float()

dataset = TensorDataset(x_tensor, y_tensor)
N = len(dataset)

train_ratio = int(data_ratio[0]*N)
val_ratio = int(data_ratio[1] * N)
test_ratio = int(N-train_ratio-val_ratio)
print("Train: ", train_ratio, ", Validation: ", val_ratio, ", Test: ", test_ratio)

train_set, val_set, test_set = random_split(dataset, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(SEED))

train_loader = DataLoader(dataset=train_set, batch_size=batch_size)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size)


Train:  3024 , Validation:  648 , Test:  648


In [17]:
# Hyper-parameters
input_dim = input_data.shape[1]
output_dim = input_data.shape[1]
k = 10
latent_dim = k * (3 + 9 + 3)         # 12 * 3
encoder_layer_sizes = [input_dim, 256, 256, latent_dim]
decoder_layer_sizes = [latent_dim, 256, 256, output_dim]
num_epochs = 100
learning_rate = 0.01
act_fn = "elu"
keep_prob = .2

# model, loss and scheduler
encoder = MLP(encoder_layer_sizes, act_fn, keep_prob, batch_size)
decoder = MLP(decoder_layer_sizes, act_fn, keep_prob, batch_size)
model = MLP_AE(encoder, decoder)

criterion = nn.MSELoss(reduction="mean")
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

print(model)

MLP_AE(
  (encoder): MLP(
    (model): Sequential(
      (fc0): Linear(in_features=315, out_features=256, bias=True)
      (act0): ELU(alpha=1.0)
      (drop1): Dropout(p=0.2, inplace=False)
      (fc1): Linear(in_features=256, out_features=256, bias=True)
      (act1): ELU(alpha=1.0)
      (drop2): Dropout(p=0.2, inplace=False)
      (fc2): Linear(in_features=256, out_features=120, bias=True)
    )
  )
  (decoder): MLP(
    (model): Sequential(
      (fc0): Linear(in_features=120, out_features=256, bias=True)
      (act0): ELU(alpha=1.0)
      (drop1): Dropout(p=0.2, inplace=False)
      (fc1): Linear(in_features=256, out_features=256, bias=True)
      (act1): ELU(alpha=1.0)
      (drop2): Dropout(p=0.2, inplace=False)
      (fc2): Linear(in_features=256, out_features=315, bias=True)
    )
  )
)


In [18]:
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

True
cuda


In [23]:
def train(model, optimizer, scheduler, num_epochs, data_loaders=None, n_epochs_no_improve=10):
    np.random.seed(SEED)
    torch.random.manual_seed(SEED)

    train_loader, val_loader, test_loader = data_loaders
    total_step = len(train_loader)
    i = 0

    train_loader_len = float(len(train_loader))
    val_loader_len = float(len(val_loader))
    test_loader_len = float(len(test_loader))

    last_avg_training_loss = 0
    min_loss = np.inf
    epochs_no_improve = 0
    best_model_after_epoch = 0

    model.cuda()
    for epoch in range(num_epochs):
        training_loss = 0
        # training
        for inputs, outputs in train_loader:
            optimizer.zero_grad()

            inputs = inputs.to(device)
            outputs = outputs.to(device)

            pred = model(inputs)
            loss = criterion(pred, outputs)
            training_loss+=loss.item()

            loss.backward()
            optimizer.step()

        scheduler.step()
        last_avg_training_loss = training_loss / train_loader_len
        print ('Epoch [{}/{}], Loss: {:.4f}'
            .format(epoch+1, num_epochs, last_avg_training_loss))

        # early stopping
        with torch.no_grad():
            val_loss = 0
            for inputs, outputs in val_loader:
                inputs = inputs.to(device)
                outputs = outputs.to(device)

                pred_val = model(inputs)
                loss_val = criterion(pred_val, outputs)
                val_loss += loss_val.item()

            val_loss /= val_loader_len
            if min_loss > val_loss:
                min_loss = val_loss
                epochs_no_improve = 0
                best_model_after_epoch = epoch

            else:
                epochs_no_improve+=1
                if epochs_no_improve > n_epochs_no_improve:
                    print("Early stopping at Epoch: ", epoch)
                    print("last training loss: {:2f}".format(last_avg_training_loss))
                    print("achieved best validation loss: {:.4f} after at Epoch {}".format(min_loss, best_model_after_epoch))
                    break

    # Testing
    with torch.no_grad():
        test_loss = 0
        for inputs, outputs in test_loader:
            inputs = inputs.to(device)
            outputs = outputs.to(device)
            pred_test = model(inputs)
            loss_test = criterion(pred_test, outputs)
            test_loss += loss_test.item()

        test_loss /= test_loader_len
        print("Test loss: {:.4f}".format(test_loss))


In [26]:
# test different initial learning rates
learning_rates = [0.1, 0.01, 0.001, 0.0001]
for lr in learning_rates:
    encoder = MLP(encoder_layer_sizes, act_fn, keep_prob, batch_size)
    decoder = MLP(decoder_layer_sizes, act_fn, keep_prob, batch_size)
    model = MLP_AE(encoder, decoder)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    train(model, optimizer, scheduler, num_epochs, (train_loader, val_loader, test_loader))

Epoch [1/100], Loss: 590055183.0022
Epoch [2/100], Loss: 1400.1388
Epoch [3/100], Loss: 759.7852
Epoch [4/100], Loss: 411.7208
Epoch [5/100], Loss: 222.1379
Epoch [6/100], Loss: 147.3039
Epoch [7/100], Loss: 64.0097
Epoch [8/100], Loss: 33.4587
Epoch [9/100], Loss: 15.0958
Epoch [10/100], Loss: 5.0964
Epoch [11/100], Loss: 1.4870
Epoch [12/100], Loss: 1.3356
Epoch [13/100], Loss: 1.2086
Epoch [14/100], Loss: 1.0914
Epoch [15/100], Loss: 0.9011
Epoch [16/100], Loss: 0.7974
Epoch [17/100], Loss: 0.7292
Epoch [18/100], Loss: 0.6777
Epoch [19/100], Loss: 0.6247
Epoch [20/100], Loss: 0.5912
Epoch [21/100], Loss: 0.5666
Epoch [22/100], Loss: 0.5716
Epoch [23/100], Loss: 0.5667
Epoch [24/100], Loss: 0.5586
Epoch [25/100], Loss: 0.5592
Epoch [26/100], Loss: 0.5558
Epoch [27/100], Loss: 0.5535
Epoch [28/100], Loss: 0.5449
Epoch [29/100], Loss: 0.5394
Epoch [30/100], Loss: 0.5376
Epoch [31/100], Loss: 0.5376
Epoch [32/100], Loss: 0.5368
Epoch [33/100], Loss: 0.5358
Epoch [34/100], Loss: 0.5351
E

In [27]:

# test different initial learning rates
learning_rates = [1e-3, 1e-4, 1e-5, 1e-6]
for lr in learning_rates:
    encoder = MLP(encoder_layer_sizes, act_fn, keep_prob, batch_size)
    decoder = MLP(decoder_layer_sizes, act_fn, keep_prob, batch_size)
    model = MLP_AE(encoder, decoder)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    train(model, optimizer, scheduler, num_epochs, (train_loader, val_loader, test_loader))

Epoch [1/100], Loss: 0.1234
Epoch [2/100], Loss: 0.0649
Epoch [3/100], Loss: 0.0664
Epoch [4/100], Loss: 0.0679
Epoch [5/100], Loss: 0.0669
Epoch [6/100], Loss: 0.0447
Epoch [7/100], Loss: 0.0428
Epoch [8/100], Loss: 0.0412
Epoch [9/100], Loss: 0.0413
Epoch [10/100], Loss: 0.0409
Epoch [11/100], Loss: 0.0393
Epoch [12/100], Loss: 0.0385
Epoch [13/100], Loss: 0.0388
Epoch [14/100], Loss: 0.0390
Epoch [15/100], Loss: 0.0389
Epoch [16/100], Loss: 0.0385
Epoch [17/100], Loss: 0.0390
Epoch [18/100], Loss: 0.0387
Epoch [19/100], Loss: 0.0390
Epoch [20/100], Loss: 0.0392
Epoch [21/100], Loss: 0.0387
Epoch [22/100], Loss: 0.0386
Epoch [23/100], Loss: 0.0386
Epoch [24/100], Loss: 0.0390
Epoch [25/100], Loss: 0.0382
Epoch [26/100], Loss: 0.0381
Epoch [27/100], Loss: 0.0393
Epoch [28/100], Loss: 0.0384
Epoch [29/100], Loss: 0.0383
Epoch [30/100], Loss: 0.0386
Epoch [31/100], Loss: 0.0384
Epoch [32/100], Loss: 0.0386
Epoch [33/100], Loss: 0.0393
Epoch [34/100], Loss: 0.0383
Epoch [35/100], Loss: 0

In [29]:
# best model
encoder = MLP(encoder_layer_sizes, act_fn, keep_prob, batch_size)
decoder = MLP(decoder_layer_sizes, act_fn, keep_prob, batch_size)
best_model = MLP_AE(encoder, decoder)

optimizer = torch.optim.AdamW(best_model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
train(best_model, optimizer, scheduler, num_epochs, (train_loader, val_loader, test_loader))

# best epoch to stop training is 59

Epoch [1/100], Loss: 0.1245
Epoch [2/100], Loss: 0.0663
Epoch [3/100], Loss: 0.0671
Epoch [4/100], Loss: 0.0485
Epoch [5/100], Loss: 0.0601
Epoch [6/100], Loss: 0.0344
Epoch [7/100], Loss: 0.0335
Epoch [8/100], Loss: 0.0336
Epoch [9/100], Loss: 0.0338
Epoch [10/100], Loss: 0.0325
Epoch [11/100], Loss: 0.0312
Epoch [12/100], Loss: 0.0310
Epoch [13/100], Loss: 0.0310
Epoch [14/100], Loss: 0.0312
Epoch [15/100], Loss: 0.0306
Epoch [16/100], Loss: 0.0307
Epoch [17/100], Loss: 0.0304
Epoch [18/100], Loss: 0.0311
Epoch [19/100], Loss: 0.0309
Epoch [20/100], Loss: 0.0302
Epoch [21/100], Loss: 0.0305
Epoch [22/100], Loss: 0.0307
Epoch [23/100], Loss: 0.0308
Epoch [24/100], Loss: 0.0300
Epoch [25/100], Loss: 0.0305
Epoch [26/100], Loss: 0.0311
Epoch [27/100], Loss: 0.0309
Epoch [28/100], Loss: 0.0309
Epoch [29/100], Loss: 0.0311
Epoch [30/100], Loss: 0.0313
Epoch [31/100], Loss: 0.0303
Epoch [32/100], Loss: 0.0309
Epoch [33/100], Loss: 0.0310
Epoch [34/100], Loss: 0.0306
Epoch [35/100], Loss: 0

In [30]:
encoder = MLP(encoder_layer_sizes, act_fn, keep_prob, batch_size)
decoder = MLP(decoder_layer_sizes, act_fn, keep_prob, batch_size)
best_model = MLP_AE(encoder, decoder)

optimizer = torch.optim.AdamW(best_model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
train(best_model, optimizer, scheduler, 59, (train_loader, val_loader, test_loader))

Epoch [1/59], Loss: 0.1220
Epoch [2/59], Loss: 0.0658
Epoch [3/59], Loss: 0.0709
Epoch [4/59], Loss: 0.0482
Epoch [5/59], Loss: 0.0541
Epoch [6/59], Loss: 0.0352
Epoch [7/59], Loss: 0.0340
Epoch [8/59], Loss: 0.0337
Epoch [9/59], Loss: 0.0339
Epoch [10/59], Loss: 0.0319
Epoch [11/59], Loss: 0.0312
Epoch [12/59], Loss: 0.0315
Epoch [13/59], Loss: 0.0315
Epoch [14/59], Loss: 0.0313
Epoch [15/59], Loss: 0.0313
Epoch [16/59], Loss: 0.0309
Epoch [17/59], Loss: 0.0316
Epoch [18/59], Loss: 0.0306
Epoch [19/59], Loss: 0.0308
Epoch [20/59], Loss: 0.0308
Epoch [21/59], Loss: 0.0307
Epoch [22/59], Loss: 0.0310
Epoch [23/59], Loss: 0.0316
Epoch [24/59], Loss: 0.0314
Epoch [25/59], Loss: 0.0307
Epoch [26/59], Loss: 0.0311
Epoch [27/59], Loss: 0.0316
Epoch [28/59], Loss: 0.0312
Epoch [29/59], Loss: 0.0304
Epoch [30/59], Loss: 0.0310
Epoch [31/59], Loss: 0.0312
Epoch [32/59], Loss: 0.0311
Epoch [33/59], Loss: 0.0312
Early stopping at Epoch:  32
last training loss: 0.031165
achieved best validation lo

In [31]:
torch.save(best_model.state_dict(), "../../models/MLP_AE_R2")

In [ ]:
data = func.load(data_path+"LOCO_R2-default-locomotion.pbz2")
data_2 = func.load(data_path+"LOCO_R2-default-locomotion-small.pbz2")
data_3 = func.load(data_path+"LOCO_R2-default-locomotion-large.pbz2")

# Prepare train data
all_data = []
data = [data, data_2, data_3]

for da in data:
    for d in da:
        d = pickle.loads(d)
        features = []
        for f in d["frames"]:
            p = np.concatenate(
                [
                    np.concatenate([jo["pos"] for jo in f]),
                    np.concatenate([jo["rotMat"].ravel() for jo in f]),
                    np.concatenate([jo["velocity"] for jo in f]),
                 ])
            features.append(p)
        all_data.append(np.vstack(features))

# input_data = np.array([np.vstack([p for p in j]) for d in all_data for j in d])
input_data = np.asarray(all_data)
input_data.reshape((-1, input_data.shape[-1]))
print(input_data.shape)

data_ratio = (.7, .15, .15) # training, validation, testing
SEED = 2021
batch_size = 10

x_tensor = torch.from_numpy(input_data).float()
y_tensor = torch.from_numpy(input_data).float()

dataset = TensorDataset(x_tensor, y_tensor)
N = len(dataset)

train_ratio = int(data_ratio[0]*N)
val_ratio = int(data_ratio[1] * N)
test_ratio = int(N-train_ratio-val_ratio)
print("Train: ", train_ratio, ", Validation: ", val_ratio, ", Test: ", test_ratio)

train_set, val_set, test_set = random_split(dataset, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(SEED))

train_loader = DataLoader(dataset=train_set, batch_size=batch_size)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size)


#%